In [ ]:
import torch
import numpy as np
import pandas as pd
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader


In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
df=pd.read_csv('/content/Nifty_50_Historical_Data_EDA.csv')
df

,Unnamed: 0.1,Unnamed: 0,Date,Price,Open,High,Low,SMA,RSI
0,6,6,2024-06-03,23263.90,23337.90,23338.70,23062.30,22775.22,64.424769
1,7,7,2024-05-31,22530.70,22568.10,22653.75,22465.10,22708.93,64.844389
2,8,8,2024-05-30,22488.65,22617.45,22705.75,22417.00,22794.21,60.978676
3,9,9,2024-05-29,22704.70,22762.75,22825.50,22685.45,22890.01,57.822527
4,10,10,2024-05-28,22888.15,22977.15,22998.55,22858.50,22868.63,57.061039
...,...,...,...,...,...,...,...,...,...
1328,1334,1334,2019-01-22,10922.75,10949.80,10949.80,10864.15,10917.41,45.967136
1329,1335,1335,2019-01-21,10961.85,10919.35,10987.45,10885.75,10910.22,49.493183
1330,1336,1336,2019-01-18,10906.95,10914.85,10928.20,10852.20,10865.37,49.634057
1331,1337,1337,2019-01-17,10905.20,10920.85,10930.65,10844.65,10842.97,50.671098


In [ ]:
df=df.drop(["Unnamed: 0.1","Unnamed: 0","Date"],axis=1)

In [ ]:
df


,Price,Open,High,Low,SMA,RSI
0,23263.90,23337.90,23338.70,23062.30,22775.22,64.424769
1,22530.70,22568.10,22653.75,22465.10,22708.93,64.844389
2,22488.65,22617.45,22705.75,22417.00,22794.21,60.978676
3,22704.70,22762.75,22825.50,22685.45,22890.01,57.822527
4,22888.15,22977.15,22998.55,22858.50,22868.63,57.061039
...,...,...,...,...,...,...
1328,10922.75,10949.80,10949.80,10864.15,10917.41,45.967136
1329,10961.85,10919.35,10987.45,10885.75,10910.22,49.493183
1330,10906.95,10914.85,10928.20,10852.20,10865.37,49.634057
1331,10905.20,10920.85,10930.65,10844.65,10842.97,50.671098


In [ ]:
close_price=df[["Price"]]


In [ ]:
import sklearn as sk
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler=sk.preprocessing.MinMaxScaler(feature_range=(0,1))
close_price=scaler.fit_transform(close_price)
close_price

array([[1.        ],
       [0.95316108],
       [0.95047481],
       ...,
       [0.21060264],
       [0.21049084],
       [0.20953899]])

In [ ]:
x_data = []
y_data = []

for i in range(100, len(close_price)):
    x_data.append(close_price[i-10:i])
    y_data.append(close_price[i])

import numpy as np
x_data, y_data = np.array(x_data), np.array(y_data)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.1,shuffle=False)

In [ ]:
X_train.shape[1],y_train.shape

(10, (1109, 1))

In [ ]:
class LSTM(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,output_size):
    super().__init__()
    self.lstm1=nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
    self.lstm2=nn.LSTM(hidden_size,64,num_layers,batch_first=True)
    self.fc=nn.Linear(64,1)

  def forward(self,x):
    out,_=self.lstm1(x)
    out,_=self.lstm2(out)
    out=self.fc(out)
    return out


In [ ]:
model=LSTM(input_size=X_train.shape[1],hidden_size=128,num_layers=3,output_size=1).to(device)

In [ ]:
optimizer=torch.optim.Adam(model.parameters(),lr=0.1)
criterion=nn.MSELoss()

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
X_train.shape

(1109, 10, 1)

In [ ]:
X_train=torch.tensor(X_train).float().to(device).squeeze(2)
y_train=torch.tensor(y_train).float().to(device).squeeze(1)
X_test=torch.tensor(X_test).float().to(device).squeeze(2)
y_test=torch.tensor(y_test).float().to(device).squeeze(1)

In [ ]:
dataset = TensorDataset(X_train, y_train)
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
y_prediction=[]
y_original=[]
epochs=100
for i in range(epochs):
  for x,y in dataloader:
    y_pred=model(x)
    y_prediction.append(y_pred)
    loss=criterion(y_pred,y)
    y_original.append(y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    with torch.no_grad():
      y_test_pred=model(X_test)
      test_loss=criterion(y_test_pred,y_test)



  if(i+1)%10==0:
    print(f'Epoch {i+1}/{epochs}, Loss: {loss.item():.4f},Test Loss: {test_loss.item():.4f}')


description:   0%|                                                           | 0/20 [15:32<?, ?it/s]


Epoch 10/100, Loss: 0.2809,Test Loss: 0.0232
Epoch 20/100, Loss: 0.1913,Test Loss: 0.0232
Epoch 30/100, Loss: 0.2374,Test Loss: 0.0232
Epoch 40/100, Loss: 0.2641,Test Loss: 0.0232
Epoch 50/100, Loss: 0.2181,Test Loss: 0.0232
Epoch 60/100, Loss: 0.2690,Test Loss: 0.0232
Epoch 70/100, Loss: 0.2582,Test Loss: 0.0232
Epoch 80/100, Loss: 0.2137,Test Loss: 0.0232
Epoch 90/100, Loss: 0.2924,Test Loss: 0.0232
Epoch 100/100, Loss: 0.2165,Test Loss: 0.0232


In [ ]:
!pip install pykan

In [ ]:
import kan
from kan import *

In [ ]:
dataset={'train_input':,'train_label':,'test_input':,'test_label':}

In [ ]:
model1 = KAN(width=[1,1,2], grid=3, k=3)

def train_acc():
    return torch.mean((torch.argmax(model1(dataset['train_input']), dim=1) == dataset['train_label']).type(dtype))

def test_acc():
    return torch.mean((torch.argmax(model1(dataset['test_input']), dim=1) == dataset['test_label']).type(dtype))

results = model1.fit(dataset, opt="LBFGS", steps=20, metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss());

description:   0%|                                                           | 0/20 [00:00<?, ?it/s]

IndexError: too many indices for tensor of dimension 2